# 1. Building code to scrap Wikipedia page

In order to obatain relevant information

In [96]:
#import library to obatain the data
import requests
#get the entire html of the url as a str
wikipedia_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [97]:
#import library for web scraping
from bs4 import BeautifulSoup
canada_data = BeautifulSoup(wikipedia_url, 'html.parser') #transform the text to html

In [98]:
#find a table in wikipedia page
canada_info_table = canada_data.find('table', class_ = 'wikitable')
canada_rows = canada_info_table.find_all('tr')

In [99]:
# extract the info ('Postcode', 'Borough', 'Neighbourhood') from the table
canada_info = []
for row in canada_rows:
    info = row.text.split('\n')[1:-1] # remove empty str (first and last items)
    canada_info.append(info)
    
canada_info[0:10]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned']]

In [136]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [100]:
import pandas as pd
import numpy as np
#transform the table into dataframe
canada_info[0][-1] = 'Neighborhood' # change to american spelling
canada_df = pd.DataFrame(canada_info[1:], columns=canada_info[0])

canada_df.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Downtown Toronto,Queen's Park


# 2. Cleaning data

Process the cells that have an assigned borough. Ignore cells with a borough that is Not assig

In [101]:
not_assigned_boroughs = canada_df.index[canada_df['Borough'] == 'Not assigned']
not_assigned_neighborhoods = canada_df.index[canada_df['Neighborhood'] == 'Not assigned']
not_assigned_neighborhoods_and_borough = not_assigned_boroughs & not_assigned_neighborhoods
print('initial rows and columns:', canada_df.shape)
print('boroughs miss value:', not_assigned_boroughs.shape[0])
print('neighborhoods miss value:', not_assigned_neighborhoods.shape[0])
print('boroughs and neighborhoods miss value:', not_assigned_neighborhoods_and_borough.shape[0])

initial rows and columns: (287, 3)
boroughs miss value: 77
neighborhoods miss value: 78
boroughs and neighborhoods miss value: 77


Then, we have to drop rows with 'Not assigned' value

In [102]:
canada_df.drop(canada_df.index[not_assigned_boroughs], inplace=True)

#reset index of dataframe
canada_df.reset_index(drop=True, inplace=True)
canada_df.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [103]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

# run this again because the indexes on the dataframe where reset
not_assigned_neighborhoods = canada_df.index[canada_df['Neighborhood'] == 'Not assigned']

for j in not_assigned_neighborhoods:
    canada_df['Neighborhood'][j] = canada_df['Borough'][j]
    
canada_df.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [104]:
group = canada_df.groupby('Postcode')
grouped_neighborhoods = group['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
grouped_boroughs = group['Borough'].apply(lambda x: set(x).pop())
grouped_df = pd.DataFrame(list(zip(grouped_boroughs.index, grouped_boroughs, grouped_neighborhoods)))
grouped_df.columns = ['Postcode', 'Borough', 'Neighborhood']

grouped_df.head(30)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [105]:
grouped_df.shape

(103, 3)

### Read the link to a csv file that has the geographical coordinates of each postal code:

In [106]:
cor_df = pd.read_csv('https://cocl.us/Geospatial_data')
cor_df.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [107]:
# adding the coordinates into our dataframe
canada_df2 = grouped_df.join(cor_df.set_index('Postal Code'), on='Postcode')
canada_df2.head(10)

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [108]:
canada_df2['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

### Create a map of Toronto with neighborhoods superimposed on top.¶


In [109]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



In [110]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [111]:
# create map of Canada using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(canada_df2['Latitude'], canada_df2['Longitude'], canada_df2['Borough'], canada_df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


### Define Foursquare Credentials and Version¶



In [112]:
CLIENT_ID = 'SJK0QCZEEP3252ONBR111YC3JPROVIYXWM0TMULH2ZJ15M04' # your Foursquare ID
CLIENT_SECRET = 'RCFJQHHYIHSGVIRHWFZ3WQXHDO0LCVF2ZBFG4IBLSUL0H3WT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SJK0QCZEEP3252ONBR111YC3JPROVIYXWM0TMULH2ZJ15M04
CLIENT_SECRET:RCFJQHHYIHSGVIRHWFZ3WQXHDO0LCVF2ZBFG4IBLSUL0H3WT


### Let's explore the first neighborhood in our dataframe.


In [113]:
canada_df2.loc[0, 'Neighborhood']

'Rouge, Malvern'

In [114]:
#Get the neighborhood's latitude and longitude values.

neighborhood_latitude = canada_df2.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = canada_df2.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = canada_df2.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url.


In [115]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=SJK0QCZEEP3252ONBR111YC3JPROVIYXWM0TMULH2ZJ15M04&client_secret=RCFJQHHYIHSGVIRHWFZ3WQXHDO0LCVF2ZBFG4IBLSUL0H3WT&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

Send the GET request and examine the resutls

In [116]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df3c7e0660a9f0025cf2a5a'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [117]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.


In [118]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [119]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


And how many venues were returned by Foursquare?


In [120]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


# Explore Neighborhoods in Toronto

Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [121]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [122]:
toronto_venues = getNearbyVenues(names=canada_df2['Neighborhood'],
                                   latitudes=canada_df2['Latitude'],
                                   longitudes=canada_df2['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

Let's check how many venues were returned for each neighborhood

In [123]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",8,8,8,8,8,8
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,55,55,55,55,55,55


Let's find out how many unique categories can be curated from all the returned venues¶

In [124]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


# Analyze Each Neighborhood

In [125]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶


In [126]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.020000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.0
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.125000,0.000000,0.000000,0.000000,0.000000,0.0
4,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
7,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.018182,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


 Let's print each neighborhood along with the top 5 most common venues

In [127]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3  Thai Restaurant  0.04
4     Burger Joint  0.03


----Agincourt----
                       venue  freq
0                     Lounge   0.2
1  Latin American Restaurant   0.2
2               Skating Rink   0.2
3             Clothing Store   0.2
4             Breakfast Spot   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0                 Park  0.33
1           Playground  0.33
2  Arts & Crafts Store  0.33
3          Yoga Studio  0.00
4          Men's Store  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.12
1          Video Store  0.12
2           Beer Store  0.12
3       Sandwich Place  0.12
4  Fried Chicken Joint  0.12


----Alderwood, Long Branch----
            venue  

                   venue  freq
0          Grocery Store  0.25
1     Athletics & Sports  0.25
2           Liquor Store  0.25
3   Gym / Fitness Center  0.25
4  Performing Arts Venue  0.00


----Downsview West----
           venue  freq
0  Grocery Store  0.33
1           Park  0.17
2          Hotel  0.17
3  Shopping Mall  0.17
4           Bank  0.17


----Downsview, North Park, Upwood Park----
                        venue  freq
0                        Park  0.33
1                      Bakery  0.33
2  Construction & Landscaping  0.33
3                 Yoga Studio  0.00
4                 Men's Store  0.00


----East Birchmount Park, Ionview, Kennedy Park----
                venue  freq
0      Discount Store  0.25
1  Chinese Restaurant  0.12
2    Department Store  0.12
3         Bus Station  0.12
4         Coffee Shop  0.12


----East Toronto----
               venue  freq
0               Park  0.50
1        Coffee Shop  0.25
2  Convenience Store  0.25
3        Yoga Studio  0.00
4        M

                venue  freq
0                Park  0.50
1               Trail  0.25
2          Playground  0.25
3   Mobile Phone Shop  0.00
4  Miscellaneous Shop  0.00


----Roselawn----
                     venue  freq
0                   Garden   0.5
1  Health & Beauty Service   0.5
2              Yoga Studio   0.0
3              Men's Store   0.0
4        Mobile Phone Shop   0.0


----Rouge, Malvern----
                       venue  freq
0       Fast Food Restaurant   0.5
1                 Print Shop   0.5
2   Mediterranean Restaurant   0.0
3         Miscellaneous Shop   0.0
4  Middle Eastern Restaurant   0.0


----Runnymede, Swansea----
                venue  freq
0                Café  0.08
1         Coffee Shop  0.08
2  Italian Restaurant  0.06
3    Sushi Restaurant  0.06
4        Dessert Shop  0.03


----Ryerson, Garden District----
                  venue  freq
0           Coffee Shop  0.09
1        Clothing Store  0.07
2        Cosmetics Shop  0.04
3  Fast Food Restaurant  0.0

Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [128]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.



In [129]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,Burger Joint,Bar,Bakery,Restaurant,Sushi Restaurant,Asian Restaurant
1,Agincourt,Latin American Restaurant,Skating Rink,Clothing Store,Lounge,Breakfast Spot,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Arts & Crafts Store,Playground,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Video Store,Beer Store,Pharmacy,Fried Chicken Joint,Fast Food Restaurant,Sandwich Place,Discount Store,Department Store
4,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Pharmacy,Sandwich Place,Dance Studio,Skating Rink,Pub,Gym,Colombian Restaurant,Curling Ice


#  Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.

In [130]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [131]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 2, 1, 1, 1, 4, 4, 4, 4], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.



In [132]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = canada_df2

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Print Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4.0,Bar,Construction & Landscaping,Moving Target,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,4.0,Electronics Store,Breakfast Spot,Medical Center,Intersection,Mexican Restaurant,Pizza Place,Rental Car Location,Spa,Doner Restaurant,Donut Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Pharmacy,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4.0,Bakery,Bank,Athletics & Sports,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Gas Station,Thai Restaurant,Women's Store,Diner


Finally, let's visualize the resulting clusters

In [138]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np # library to handle data in a vectorized manner

In [134]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [145]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float

# Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. 

Cluster 1

In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,0.0,Cafeteria,Dog Run,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store,Dance Studio


Cluster 2


In [147]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1.0,Fast Food Restaurant,Print Shop,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
3,Scarborough,1.0,Coffee Shop,Korean Restaurant,Pharmacy,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
13,Scarborough,1.0,Pizza Place,Bank,Noodle House,Shopping Mall,Pharmacy,Italian Restaurant,Chinese Restaurant,Fast Food Restaurant,Fried Chicken Joint,Thai Restaurant
15,Scarborough,1.0,Fast Food Restaurant,Chinese Restaurant,Thrift / Vintage Store,Coffee Shop,Grocery Store,Gym,Breakfast Spot,Sandwich Place,Pharmacy,Pizza Place
17,North York,1.0,Golf Course,Pool,Fast Food Restaurant,Dog Run,Mediterranean Restaurant,Athletics & Sports,Comic Shop,Department Store,Eastern European Restaurant,Dumpling Restaurant
24,North York,1.0,Coffee Shop,Pharmacy,Pizza Place,Butcher,Discount Store,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop
28,North York,1.0,Coffee Shop,Middle Eastern Restaurant,Frozen Yogurt Shop,Supermarket,Sushi Restaurant,Deli / Bodega,Fast Food Restaurant,Restaurant,Pizza Place,Pharmacy
33,North York,1.0,Athletics & Sports,Grocery Store,Gym / Fitness Center,Liquor Store,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
35,East York,1.0,Pizza Place,Fast Food Restaurant,Intersection,Bank,Athletics & Sports,Gym / Fitness Center,Gastropub,Pharmacy,Bus Line,Dessert Shop
88,Etobicoke,1.0,Flower Shop,Restaurant,Liquor Store,Coffee Shop,Café,Gym,Sandwich Place,Pharmacy,American Restaurant,Bakery


cluster 3

In [148]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,2.0,Park,Arts & Crafts Store,Playground,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run
21,North York,2.0,Park,Piano Bar,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Women's Store
23,North York,2.0,Park,Bank,Convenience Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Deli / Bodega
25,North York,2.0,Park,Food & Drink Shop,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Ethiopian Restaurant
30,North York,2.0,Park,Airport,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
40,East York,2.0,Park,Convenience Store,Coffee Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Women's Store
44,Central Toronto,2.0,Park,Gym / Fitness Center,Bus Line,Swim School,Dumpling Restaurant,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dance Studio
50,Downtown Toronto,2.0,Park,Playground,Trail,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Eastern European Restaurant,Cupcake Shop
64,Central Toronto,2.0,Park,Sushi Restaurant,Trail,Jewelry Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
74,York,2.0,Park,Fast Food Restaurant,Market,Women's Store,General Travel,General Entertainment,Dumpling Restaurant,Drugstore,Donut Shop,Gift Shop


cluster 4

In [151]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,3.0,Playground,Women's Store,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop


cluster 5

In [150]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,4.0,Bar,Construction & Landscaping,Moving Target,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
2,Scarborough,4.0,Electronics Store,Breakfast Spot,Medical Center,Intersection,Mexican Restaurant,Pizza Place,Rental Car Location,Spa,Doner Restaurant,Donut Shop
4,Scarborough,4.0,Bakery,Bank,Athletics & Sports,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Gas Station,Thai Restaurant,Women's Store,Diner
6,Scarborough,4.0,Discount Store,Chinese Restaurant,Convenience Store,Hobby Shop,Department Store,Bus Station,Coffee Shop,Dumpling Restaurant,Drugstore,Eastern European Restaurant
7,Scarborough,4.0,Bus Line,Bakery,Metro Station,Park,Bus Station,Soccer Field,Fast Food Restaurant,Intersection,Diner,Dessert Shop
8,Scarborough,4.0,Motel,American Restaurant,Women's Store,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
9,Scarborough,4.0,Skating Rink,College Stadium,Café,General Entertainment,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
10,Scarborough,4.0,Indian Restaurant,Furniture / Home Store,Vietnamese Restaurant,Pet Store,Chinese Restaurant,Thrift / Vintage Store,Women's Store,Deli / Bodega,Department Store,Dessert Shop
11,Scarborough,4.0,Auto Garage,Breakfast Spot,Bakery,Sandwich Place,Middle Eastern Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Eastern European Restaurant,Ethiopian Restaurant
12,Scarborough,4.0,Latin American Restaurant,Skating Rink,Clothing Store,Lounge,Breakfast Spot,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
